In [1]:
import pandas as pd
import xarray as xr
import rasterio as rio
import numpy as np
import rioxarray
from glob import glob
import geopandas as gpd

In [2]:
md_df = pd.read_csv('md_processing_decisions.csv').iloc[:, 0:9]
md_df['survey_name'] = md_df['survey_name'].str.strip()

In [5]:
md_area_dic = {}
moving_area_dic = {}
asc_veloc_dic = {}
des_veloc_dic = {}
ud_veloc_dic = {}

asc_veloc_75_dic = {}
des_veloc_75_dic = {}
ud_veloc_75_dic = {}

asc_veloc_max_dic = {}
des_veloc_max_dic = {}
ud_veloc_max_dic = {}

for i, row in md_df.iterrows():
    print(row.survey_name)
    md_gdf = gpd.read_file(f'/home/jovyan/repos/incubator_moraine_dam_analysis/mapping/polygons/{row.survey_name}_md.shp')
    md_area_dic[f'{row.survey_name}'] = md_gdf.area.item()
    try:
        moving_gdf = gpd.read_file(f'/home/jovyan/repos/incubator_moraine_dam_analysis/mapping/polygons/{row.survey_name}_moving.shp').dissolve()
    except:
        asc_veloc_dic[f'{row.survey_name.strip()}'] = np.nan
        des_veloc_dic[f'{row.survey_name.strip()}'] = np.nan
        ud_veloc_dic[f'{row.survey_name.strip()}'] = np.nan
        continue
    
    moving_area_dic[f'{row.survey_name}'] = moving_gdf.area.item()
    asc_ds = xr.open_dataset(f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{row.survey_name}/{row.survey_name.strip()}_asc_median_velocity.tif').squeeze()
    des_ds = xr.open_dataset(f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{row.survey_name}/{row.survey_name.strip()}_des_median_velocity.tif').squeeze()
    ud_ds = xr.open_dataset(f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{row.survey_name}/{row.survey_name.strip()}_ud_median_velocity.tif').squeeze()
    asc_veloc_dic[f'{row.survey_name.strip()}'] = asc_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.median().item()
    des_veloc_dic[f'{row.survey_name.strip()}'] = des_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.median().item()
    ud_veloc_dic[f'{row.survey_name.strip()}'] = ud_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.median().item()

    asc_veloc_75_dic[f'{row.survey_name.strip()}'] = asc_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.quantile(q=0.75).item()
    des_veloc_75_dic[f'{row.survey_name.strip()}'] = des_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.quantile(q=0.75).item()
    ud_veloc_75_dic[f'{row.survey_name.strip()}'] = ud_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.quantile(q=0.25).item()

    asc_veloc_max_dic[f'{row.survey_name.strip()}'] = asc_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.max().item()
    des_veloc_max_dic[f'{row.survey_name.strip()}'] = des_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.max().item()
    ud_veloc_max_dic[f'{row.survey_name.strip()}'] = ud_ds.rio.clip(moving_gdf.geometry, crs=moving_gdf.crs).band_data.min().item()

rolpa
barun
imja
lumding
chamlang_south
thugali
nagma
chamlang_north
muli_tal
barun_pokhari
east_hongu_1
birendra
mera
upper_barun
gakal_gl_0008
kotam_gl_0111
east_hongu_2
tallo_kekyap
kekyap
dig
sabai
kodud_gl_0205
hongu_1


In [6]:
md_df['area'] = md_df['survey_name'].map(md_area_dic)
md_df['moving_area'] = md_df['survey_name'].map(moving_area_dic)
md_df['asc_median_veloc'] = md_df['survey_name'].map(asc_veloc_dic)
md_df['des_median_veloc'] = md_df['survey_name'].map(des_veloc_dic)
md_df['ud_median_veloc'] = md_df['survey_name'].map(ud_veloc_dic)

md_df['asc_75_veloc'] = md_df['survey_name'].map(asc_veloc_75_dic)
md_df['des_75_veloc'] = md_df['survey_name'].map(des_veloc_75_dic)
md_df['ud_75_veloc'] = md_df['survey_name'].map(ud_veloc_75_dic)

md_df['asc_max_veloc'] = md_df['survey_name'].map(asc_veloc_max_dic)
md_df['des_max_veloc'] = md_df['survey_name'].map(des_veloc_max_dic)
md_df['ud_max_veloc'] = md_df['survey_name'].map(ud_veloc_max_dic)

In [16]:
md_df['moving_proportion'] = md_df['moving_area']/md_df['area']

In [7]:
md_df

,icimod_id,survey_name,asc_burst,des_burst,lat_lot,significant_snow,highcorr_period,lowcorr_period,snow_period,area,moving_area,asc_median_veloc,des_median_veloc,ud_median_veloc,asc_75_veloc,des_75_veloc,ud_75_veloc,asc_max_veloc,des_max_veloc,ud_max_veloc
0,kotak_gl_0009,rolpa,012_023790_IW1,121_258662_IW2,"27.862, 86.477",yes,100-160,220-280,340-100,4.191709e+05,10129.759178,-0.323984,0.029579,0.197150,-0.035126,0.059038,0.011075,0.046149,0.119134,-0.064706
1,koaru_gl_0009,barun,012_023789_IW1,121_258662_IW1,"27.798, 87.097",yes,120-180,220-280,340-120,7.910867e+05,247760.493792,0.028801,0.066983,-0.061239,0.098959,0.119038,-0.138696,0.348387,0.395695,-0.383991
2,kodud_gl_0184,imja,012_023790_IW1,121_258661_IW2,"27.899, 86.923",no,0-100,220-280,NaN,6.215257e+05,303357.872967,0.099310,0.066936,-0.119386,0.204887,0.104285,-0.203606,0.561895,0.314191,-0.399108
3,kodud_gl_0036,lumding,012_023789_IW1,121_258662_IW2,"27.779, 86.614",yes,160-200,230-290,320-160,1.878586e+05,22345.184143,0.069863,0.029382,-0.064481,0.116844,0.048991,-0.110613,0.169191,0.093791,-0.158218
4,kodud_gl_0242,chamlang_south,012_023789_IW1,121_258662_IW2,"27.755, 86.96",no,0-100,230-290,NaN,8.688254e+05,235724.486827,0.021180,0.037528,-0.036560,0.037812,0.065928,-0.062437,0.126518,0.176878,-0.177187
5,gamar_gl_0018,thugali,085_180592_IW1,019_039569_IW2,"28.489, 84.486",yes,120-180,220-280,340-120,3.336042e+05,130973.466724,0.029214,0.222565,-0.100752,0.105544,0.282405,-0.205847,0.271140,0.448431,-0.369237
6,kotam_gl_0135,nagma,012_023788_IW2,048_101863_IW3,"27.87, 87.867",yes,130-200,240-300,340-130,2.757860e+05,68743.270436,0.005021,0.043231,-0.034856,0.016916,0.076234,-0.056670,0.040210,0.431732,-0.267807
7,kodud_gl_0241,chamlang_north,012_023789_IW1,121_258662_IW2,"27.784, 86.957",no,0-100,230-290,NaN,9.050762e+05,252010.634840,0.031836,0.007137,-0.028703,0.054492,0.025011,-0.052780,0.115921,0.450102,-0.270729
8,gakal_gl_0004,muli_tal,158_337392_IW2,092_196365_IW1,"28.888, 83.528",yes,120-200,220-280,260-120,1.690491e+04,14147.264385,0.055597,0.039605,-0.060230,0.075176,0.056513,-0.091985,0.140086,0.101657,-0.113254
9,kodud_gl_0012,barun_pokhari,012_023789_IW1,121_258662_IW1,"27.845, 87.082",yes,120-180,230-290,340-120,1.079131e+05,30338.100500,0.030332,0.033173,-0.036197,0.044179,0.053151,-0.053712,0.089205,0.100531,-0.102698
